In [ ]:
from radas import run_experiment, get_ray_results_dir  # pyright: ignore[reportMissingImports]
import ray.tune as tune
from DiscPC_AdamW import trainable
import torch
import pandas as pd
import numpy as np


results = await run_experiment(
    user_name="mengfan",
    experiment_name="DiscPC的baseline实验",
    param_space=dict(
        seed = tune.grid_search([i for i in range(10)]),  
        epochs = tune.grid_search([50]),
        lr_x = tune.grid_search([1e-3,2e-3,3e-3,4e-3,5e-3,6e-3,7e-3,8e-3,9e-3,1e-2,2e-2,3e-2,4e-2,5e-2]),
        use_bias = tune.grid_search([True,False]),

    ),
    # IMPORTANT: when using cluster, your trainable cannot be defined in this script, but has to be defined in a separate file, and imported in this script
    trainable=trainable,
    # When use cluster, local_storage_path needs to be specified in this way
    local_storage_path=get_ray_results_dir(),
    # Specify the cluster to use
    run_with="cluster:atol-gpu-5090",
    # Specify gram per trial
    cpu_per_trial=1,
    # If the gram_per_trial you set is larger than GRAM of a single GPU, you will be allocated with more than one GPUs
    # to fit your need. But in this case, you will need to handle how do you want to distribute your workload to these multiple GPUs
    # To check GRAM of a single GPU, use the following code:
    # from radas.clusters import gram_per_gpu
    #
    runtime_env={
        "pip": ["emoji","scikit-learn"],
        # for more runtime_env specifications, see https://docs.ray.io/en/latest/ray-core/handling-dependencies.html#api-reference
    },
    #
    instant_gpu_allocation=1,
)

In [ ]:
results["df"]
df = results["df"]

In [ ]:
# 2. 一键保存为pkl文件（index=False 不存行索引，更简洁）
df.to_pickle("discpc_baseline_AdamW.pkl", protocol=4)  # protocol=4 兼容多数Python版本，读写更快